In [1]:
import numpy as np
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

**Sample Data**

In [10]:
# Stride and Pad
(S, P) = (2, 1)
# Input Channels, HeightIn, WidthIn
(C, HI, WI) = (3, 5, 5)
# Filters, HeightFilter, WidthFilter
(F, HF, WF) = (2, 3, 3)
# Output image HeightOut, WidthOut
(HO, WO) = ((HI - HF + 2*P)/S + 1, (WI - WF + 2*P)/S + 1) 
print "HO, WO", HO, WO

x= np.empty((C, HI, WI))
x[0, :, :] = np.array([
    [2, 0, 2, 2, 0], 
    [1, 2, 2, 2, 2], 
    [0, 0, 1, 0, 0], 
    [2, 1, 0, 2, 1], 
    [0, 1, 0, 1, 2], 
])

x[1, :, :] = np.array([
    [2, 0, 1, 2, 0], 
    [2, 0, 2, 0, 1], 
    [2, 1, 1, 2, 1], 
    [2, 1, 1, 2, 2], 
    [2, 0, 0, 0, 2], 
])

x[2, :, :]  = np.array([
    [2, 2, 2, 2, 0], 
    [1, 0, 0, 1, 2], 
    [2, 1, 2, 2, 1], 
    [2, 0, 2, 2, 1], 
    [1, 0, 0, 2, 1], 
])

# Filters, Channels, HF, WF
w = np.empty((2, 3, 3, 3))
w[0, 0, :, :] = np.array([
        [0, 0, -1],
        [0, 1, 0],
        [1, 0, 0]
    ])
w[0, 1, :, :] = np.array([
        [1, 1, 1],
        [-1, 1, 1],
        [1, 0, -1]
    ])
w[0, 2, :, :] = np.array([
        [0, 1, -1],
        [0, 1, 1],
        [0, -1, 1]
    ])

w[1, 0, :, :] = np.array([
        [-1, 0, 1],
        [0, 1, -1],
        [0, 1, 1]
    ])
w[1, 1, :, :] = np.array([
        [1, -1, 0],
        [1, 0, 1],
        [0, 1, -1]
    ])
w[1, 2, :, :] = np.array([
        [-1, -1, 1],
        [0, -1, -1],
        [-1, 0, 0]
    ])
b = np.array([1, 0])

correct_out = np.array([
        [[8, 13, -1],
        [5, 7, 7],
        [8, 5, 11]],
        [[3, 4, 4],
        [1, 0, -4],
        [-5, -2, -4]],
    ])

HO, WO 3 3


** Forward pass **

In [14]:
x_pad = np.pad(x, ((0, 0), (P, P), (P, P)), 'constant')
out = np.empty((F, HO, WO))
for f in range (F):
    for i in range(HO):
      for j in range(WO):
            out[f, i, j] = np.sum(x_pad[:, i*S:i*S+HF, j*S:j*S+WF] * w[f]) + b[f]

print 'Forward pass error'
print 'error: ', rel_error(out, correct_out)
# print out
# print correct_out

Forward pass error
error:  0.0


** Forward pass in steps **

In [15]:
def pad_x(x, P):
    """
    Pad x with P zeros before and after image dimensions
    Input 
        - x (C, H, W)
    Output
        - x(C, H + 2P, W + 2P)
    """
    return np.pad(x, ((0, 0), (P, P), (P, P)), 'constant')


def distribute_x(x, HF, WF, S):
    '''
    Distribute input image to convolution ready sub-images.  
    Make a filter ready image for each filter and output image dimension 
    Inputs:
    - x (C, H, W) input image
    Output
    - out (F, HO, WO, C, HF, WF)
    -- F number of filters
    -- HO, WO, output dimensions
    -- C Input channels
    -- HF WF filter dimensions
    '''
    (C, HI, WI) = x.shape
    HO = 1 + (HI - HF) / S
    WO = 1 + (WI - WF) / S
    out = np.empty((F, HO, WO, C, HF, WF))

    for f in range(F):
        for i in range(HO):
            for j in range(WO):
                out[f, i, j] = x[:, i*S:i*S+HF, j*S:j*S+WF]
    return out

def multiply_w(x, w):
    '''
    Apply filters to (C, HF, WF) sections
    Inputs
    - x (F, HO, WO, C, HF, WF)
    - w (F, C, HF, WF)
    Returns
    - out (F, HO, WO, C, HF, WF)
    '''
    out = np.empty((F, HO, WO, C, HF, WF))
    for f in range(F):
        for i in range(HO):
            for j in range(WO):
                out[f, i, j] = x[f, i, j] * w[f]
    return out
    
def sum_filter(x):
    '''
    For each out pixel sum up weighted x*w over (C, HF, WF)
    Input
    - x (F, HO, WO, C, HF, WF)
    Output
    - out (F, HO, WO)
    '''
    out = np.empty((F, HO, WO))
    for f in range(F):
        for i in range(HO):
            for j in range(WO):
                out[f, i, j] = np.sum(x[f, i, j])
    return out

def add_bias(x, b):
    '''
    Apply the bias terms
    Inputs
    - x (F, HO, WO)
    - b (F,)
    '''
    out = np.empty_like(x)
    for f in range(F):
        out[f] = x[f] + b[f]
    return out


### Test back prop on pad

In [6]:
def padx(x, P):
    """
    Pad x with P zeros before and after image dimensions
    Input 
        - x (C, H, W)
    Output
        - x(C, H + 2P, W + 2P)
    """
    return np.pad(x, ((0, 0), (P, P), (P, P)), 'constant')

dout = np.random.randn(C, HI + 2 st* P, WI + 2 * P)

# evaluate numerical gradient
dx_num = eval_numerical_gradient_array(lambda x: padx(x, P), x, dout)
# print "dx_num\n", dx_num

# evalute gradient via backpropagation
dx = dout[:, 1:HI + 2 * P - 1, 1:WI + 2 * P - 1]
# print "dx\n", dx

print 'Testing pad function'
print 'dx error: ', rel_error(dx, dx_num)


Testing pad function
dx error:  3.27561298137e-12


### test backprop on distribute

In [14]:

x_pad = padx(x, P)   
d = distribute(x_pad, HF, WF, S)
print d.shape
    

    

(2, 3, 3, 3, 3, 3)
